In [1]:
import os
from openai import OpenAI

from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential

from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    ToolMessage,
    AssistantMessage
)



In [37]:

endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT")
model_name  = os.getenv("AZURE_OPENAI_DEPLOYMENT")
project_connect = os.getenv("AZURE_AI_PROJECT_CONN_STR")

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(os.getenv("AZURE_INFERENCE_SDK_ENDPOINT_KEY")),
    # credential=DefaultAzureCredential(),
    credential_scopes=["https://cognitiveservices.azure.com/.default"],

    )


messages = [
    UserMessage(content="1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?"),
]

In [38]:
from azure.ai.projects import AIProjectClient
from azure.monitor.opentelemetry import configure_azure_monitor
import sys


project_client = AIProjectClient.from_connection_string(
    conn_str=project_connect,
    credential=DefaultAzureCredential()
)

project_client.telemetry.get_connection_string()
project_client.telemetry.enable(destination=sys.stdout)

In [39]:
print(messages[0].content)

try:
    response = client.complete(
        messages = messages,
        model = model_name,
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error: {e}")

1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?
If shirts are being dried under the same conditions and at the same time, **3 shirts will still take 1 hour to dry**—not more. Drying time doesn’t accumulate based on the number of shirts being dried simultaneously, as long as the drying process can accommodate them all at once.


In [40]:
import os
from opentelemetry import trace

# Install opentelemetry with command "pip install opentelemetry-sdk".
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, CompletionsFinishReason
from azure.core.credentials import AzureKeyCredential


In [7]:
# [START trace_setting]
from azure.core.settings import settings

settings.tracing_implementation = "opentelemetry"
# [END trace_setting]

In [41]:
project_client.telemetry.get_connection_string()
project_client.telemetry.enable(destination=sys.stdout)

In [42]:
# https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/inference/sample_chat_completions_with_azure_openai_client_and_azure_monitor_tracing.py
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()

# Enable additional instrumentations for openai and langchain
# which are not included by Azure Monitor out of the box
project_client.telemetry.enable()
configure_azure_monitor(connection_string=application_insights_connection_string)

In [43]:
with project_client.inference.get_azure_openai_client(api_version="2024-06-01") as client:

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": "How many feet are in a mile?",
            },
        ],
    )

    print(response.choices[0].message.content)

There are **5,280 feet** in a mile.


In [47]:
with project_client.inference.get_chat_completions_client() as client:
    response = client.complete(
        messages=[
            {
                "role": "user",
                "content": "Hello there",
            },
        ],
        model=model_name,
    )

    print(response.choices[0].message.content)
    # print(client)

Hello! 😊 How can I assist you today?


In [21]:
from azure.ai.projects.models import ConnectionType

[ct for ct in ConnectionType]

[<ConnectionType.AZURE_OPEN_AI: 'AzureOpenAI'>,
 <ConnectionType.SERVERLESS: 'Serverless'>,
 <ConnectionType.AZURE_BLOB_STORAGE: 'AzureBlob'>,
 <ConnectionType.AZURE_AI_SERVICES: 'AIServices'>,
 <ConnectionType.AZURE_AI_SEARCH: 'CognitiveSearch'>,
 <ConnectionType.API_KEY: 'ApiKey'>]

In [ ]:
conn = project_client.connections.get_default(connection_type=ConnectionType.AZURE_OPEN_AI)

print(conn)

{
 "name": "ai-mutazaaieastus2054278078742_aoai",
 "id": "/subscriptions/e9b9c71e-d2ba-4086-a5ed-2923e15962d9/resourceGroups/rg-mutaza-5102_ai/providers/Microsoft.MachineLearningServices/workspaces/ai-inference-eastus2/connections/ai-mutazaaieastus2054278078742_aoai",
 "authentication_type": "ApiKey",
 "connection_type": "AzureOpenAI",
 "endpoint_url": "https://ai-mutazaaieastus2054278078742.openai.azure.com",
 "key": null
 "token_credential": null
}



<function azure.ai.projects.telemetry._trace_function.trace_function.<locals>.decorator(func: Callable) -> Callable>

In [8]:
# # Setup tracing to console
# # Requires opentelemetry-sdk
# span_exporter = ConsoleSpanExporter()
# tracer_provider = TracerProvider()
# tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))
# trace.set_tracer_provider(tracer_provider)


In [ ]:
configure_azure_monitor(connection_string=project_connect)

In [ ]:
with project_client.inference.get_chat_completions_client() as chat_client:

    response = chat_client.complete(
        model=model_name, messages=[UserMessage(content="How many feet are in a mile?")]
    )

    print(response.choices[0].message.content)

In [ ]:
project_client.telemetry.get_connection_string()

In [ ]:
project_client.telemetry.enable(destination=project_client.telemetry.get_connection_string())

In [ ]:
print(project_client.telemetry)

In [ ]:
project_client.telemetry.enable(destination="azure-ai-projects")

In [ ]:
with project_client.inference.get_chat_completions_client() as chat_client:

    response = chat_client.complete(
        model=model_name, messages=[UserMessage(content="How many feet are in a mile?")]
    )

    print(response.choices[0].message.content)